# Deepstream Test4 description
--------------------------------------------------------------------------------------
This sample builds on top of the deepstream-test1 sample to demonstrate how to:

* Use "nvmsgconv" and "nvmsgbroker" plugins in the pipeline.
* Create NVDS_META_EVENT_MSG type of meta and attach to buffer.
* Use NVDS_META_EVENT_MSG for different types of objects e.g. vehicle, person etc.
* copy / free functions if meta data is extended through "extMsg" field have been moved out of the deepstream_test4 python app.
These copy and free functions are available for using/extending inside bindschema.cpp as event_msg_meta_copy_func() and
event_msg_meta_release_func() respectively.

"nvmsgconv" plugin uses NVDS_META_EVENT_MSG type of metadata from the buffer
and generates the "DeepStream Schema" payload in Json format. Static properties
of schema are read from configuration file in the form of key-value pair.
Check dstest4_msgconv_config.txt for reference. Generated payload is attached
as NVDS_META_PAYLOAD type metadata to the buffer.

"nvmsgbroker" plugin extracts NVDS_META_PAYLOAD type of metadata from the buffer
and sends that payload to the server using protocol adaptor APIs.

Generating custom metadata for different type of objects:
In addition to common fields provided in NvDsEventMsgMeta structure, user can
also create custom objects and attach to buffer as NVDS_META_EVENT_MSG metadata.
To do that NvDsEventMsgMeta provides "extMsg" and "extMsgSize" fields. User can
create custom structure, fill that structure and assign the pointer of that
structure as "extMsg" and set the "extMsgSize" accordingly.
If custom object contains fields that can't be simply mem copied then user should
also provide/extend the functions to copy - event_msg_meta_copy_func() and free - event_msg_meta_release_func() those objects.

# Prerequisites

   * **DeepStream SDK 7.0**

To setup and install DeepStream 7.0, please follow the steps at https://developer.nvidia.com/deepstream-getting-started
    
   * **DeepStream Python Apps**

To install DeepStream Python Apps, follow the instructions in this repo: https://github.com/NVIDIA-AI-IOT/deepstream_python_apps

   * [**Jupyter Notebook**](https://jupyter.org/install)

To install jupyter notebook, run the following command:

```
pip3 install notebook
```  
<br>
* **Build dependencies with installation instructions:**

&nbsp;&nbsp;&nbsp;&nbsp;__librdkafka__

  Note that for using TLS/SSL security, make sure to build librdkafka with
  SSL suport enabled by using the enable_ssl option while running
  'configure', as shown below. Also note that these are the same steps as setup in [Deepstream Quickstart Guide](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_Quickstart.html#id1). If already previously completed, please skip this step. 
```
  git clone https://github.com/confluentinc/librdkafka.git
  cd librdkafka
  git checkout tags/v2.2.0
  ./configure --enable-ssl
  make
  sudo make install
  sudo cp /usr/local/lib/librdkafka* /opt/nvidia/deepstream/deepstream/lib/
  sudo ldconfig
```
NOTE: To compile the sources, run make with "sudo" or root permission.

&nbsp;&nbsp;&nbsp;&nbsp;__glib 2.0__
   
```
  apt-get install libglib2.0 libglib2.0-dev 
```
<br>
  
&nbsp;&nbsp;&nbsp;&nbsp;__jansson__
    
```
  apt-get install  libjansson4  libjansson-dev
``` 
<br>
  
&nbsp;&nbsp;&nbsp;&nbsp;__ssl__
   
```
  apt-get install libssl-dev
 ```
<br>

* (Optional): Install & setup kafka broker on your machine & create topic(s). See instructions here: https://kafka.apache.org/quickstart (These are the same steps as deepstream-test4)

 1. Install kafka server
 2. In a terminal tab, navigate to where the kafka server is installed. Run
```bin/zookeeper-server-start.sh config/zookeeper.properties```
 3. Open another tab in terminal. Run ``bin/kafka-server-start.sh config/server.properties`` from the same directory where kafka is installed.
 4. Open another tab in terminal. Run ``bin/kafka-topics.sh --create --topic topic --bootstrap-server localhost:9092``. You may need to delete an old topic (``./bin/kafka-topics.sh --delete --topic topic --bootstrap-server localhost:9092``) and recreate if the topic is already created. Again, run in same directory where kafka is installed.

# Importing necessary libraries

In [ ]:
import sys

sys.path.append('../')
sys.path.append('../apps/')
import gi

gi.require_version('Gst', '1.0')
from gi.repository import GLib, Gst
import sys
from optparse import OptionParser
from common.platform_info import PlatformInfo
from common.bus_call import bus_call
from common.utils import long_to_uint64
import pyds

# Declaring class label ids and other meta data requirements

## Example: 
cfg_file = "../apps/deepstream-test4/cfg_kafka.txt"
input_file = "/opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264"
proto_lib = "/opt/nvidia/deepstream/deepstream/lib/libnvds_kafka_proto.so"
conn_str = "localhost;2181;testTopic"
topic = "topic"
no_display = False
schema_type = 0    

In [ ]:
MAX_DISPLAY_LEN = 64
MAX_TIME_STAMP_LEN = 32
PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3
MUXER_OUTPUT_WIDTH = 1920
MUXER_OUTPUT_HEIGHT = 1080
MUXER_BATCH_TIMEOUT_USEC = 33000
cfg_file = "../apps/deepstream-test4/cfg_kafka.txt"
input_file = "/opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264"
proto_lib = "/opt/nvidia/deepstream/deepstream/lib/libnvds_kafka_proto.so"
conn_str = "localhost;2181;testTopic" # make sure to change to your own connection string
topic = "topic"
no_display = False
schema_type = 0    

# Necessary config files and meta data class labels

In [ ]:
PGIE_CONFIG_FILE = "../apps/deepstream-test4/dstest4_pgie_config.txt"
MSCONV_CONFIG_FILE = "../apps/deepstream-test4/dstest4_msgconv_config.txt"
pgie_classes_str = ["Vehicle", "TwoWheeler", "Person", "Roadsign"]

# Generating meta data 

In [ ]:
def generate_vehicle_meta(data):
    obj = pyds.NvDsVehicleObject.cast(data)
    obj.type = "sedan"
    obj.color = "blue"
    obj.make = "Bugatti"
    obj.model = "M"
    obj.license = "XX1234"
    obj.region = "CA"
    return obj


def generate_person_meta(data):
    obj = pyds.NvDsPersonObject.cast(data)
    obj.age = 45
    obj.cap = "none"
    obj.hair = "black"
    obj.gender = "male"
    obj.apparel = "formal"
    return obj

# Generating event meta data

In [ ]:
def generate_event_msg_meta(data, class_id):
    meta = pyds.NvDsEventMsgMeta.cast(data)
    meta.sensorId = 0
    meta.placeId = 0
    meta.moduleId = 0
    meta.sensorStr = "sensor-0"
    meta.ts = pyds.alloc_buffer(MAX_TIME_STAMP_LEN + 1)
    pyds.generate_ts_rfc3339(meta.ts, MAX_TIME_STAMP_LEN)

    # This demonstrates how to attach custom objects.
    # Any custom object as per requirement can be generated and attached
    # like NvDsVehicleObject / NvDsPersonObject. Then that object should
    # be handled in payload generator library (nvmsgconv.cpp) accordingly.
    if class_id == PGIE_CLASS_ID_VEHICLE:
        meta.type = pyds.NvDsEventType.NVDS_EVENT_MOVING
        meta.objType = pyds.NvDsObjectType.NVDS_OBJECT_TYPE_VEHICLE
        meta.objClassId = PGIE_CLASS_ID_VEHICLE
        obj = pyds.alloc_nvds_vehicle_object()
        obj = generate_vehicle_meta(obj)
        meta.extMsg = obj
        meta.extMsgSize = sys.getsizeof(pyds.NvDsVehicleObject)
    if class_id == PGIE_CLASS_ID_PERSON:
        meta.type = pyds.NvDsEventType.NVDS_EVENT_ENTRY
        meta.objType = pyds.NvDsObjectType.NVDS_OBJECT_TYPE_PERSON
        meta.objClassId = PGIE_CLASS_ID_PERSON
        obj = pyds.alloc_nvds_person_object()
        obj = generate_person_meta(obj)
        meta.extMsg = obj
        meta.extMsgSize = sys.getsizeof(pyds.NvDsPersonObject)
    return meta

# Adding a probe to get meta data

## osd_sink_pad_buffer_probe  will extract metadata received on OSD sink pad
## and update params for drawing rectangle, object information etc.

## IMPORTANT NOTE:
#### a) probe() callbacks are synchronous and thus holds the buffer
####      (info.get_buffer()) from traversing the pipeline until user return.
#### b) loops inside probe() callback could be costly in python.
####   So users shall optimize according to their use-case.

In [ ]:
def osd_sink_pad_buffer_probe(pad, info, u_data):
    frame_number = 0
    # Intiallizing object counter with 0.
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE: 0,
        PGIE_CLASS_ID_PERSON: 0,
        PGIE_CLASS_ID_BICYCLE: 0,
        PGIE_CLASS_ID_ROADSIGN: 0
    }
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    if not batch_meta:
        return Gst.PadProbeReturn.OK
    l_frame = batch_meta.frame_meta_list
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            # The casting is done by pyds.NvDsFrameMeta.cast()
            # The casting also keeps ownership of the underlying memory
            # in the C code, so the Python garbage collector will leave
            # it alone.
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            continue
        is_first_object = True

        # Short example of attribute access for frame_meta:
        # print("Frame Number is ", frame_meta.frame_num)
        # print("Source id is ", frame_meta.source_id)
        # print("Batch id is ", frame_meta.batch_id)
        # print("Source Frame Width ", frame_meta.source_frame_width)
        # print("Source Frame Height ", frame_meta.source_frame_height)
        # print("Num object meta ", frame_meta.num_obj_meta)

        frame_number = frame_meta.frame_num
        l_obj = frame_meta.obj_meta_list
        while l_obj is not None:
            try:
                obj_meta = pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                continue

            # Update the object text display
            txt_params = obj_meta.text_params

            # Set display_text. Any existing display_text string will be
            # freed by the bindings module.
            txt_params.display_text = pgie_classes_str[obj_meta.class_id]

            obj_counter[obj_meta.class_id] += 1

            # Font , font-color and font-size
            txt_params.font_params.font_name = "Serif"
            txt_params.font_params.font_size = 10
            # set(red, green, blue, alpha); set to White
            txt_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)

            # Text background color
            txt_params.set_bg_clr = 1
            # set(red, green, blue, alpha); set to Black
            txt_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)

            # Ideally NVDS_EVENT_MSG_META should be attached to buffer by the
            # component implementing detection / recognition logic.
            # Here it demonstrates how to use / attach that meta data.
            if is_first_object and (frame_number % 30) == 0:
                # Frequency of messages to be send will be based on use case.
                # Here message is being sent for first object every 30 frames.

                user_event_meta = pyds.nvds_acquire_user_meta_from_pool(
                    batch_meta)
                if user_event_meta:
                    # Allocating an NvDsEventMsgMeta instance and getting
                    # reference to it. The underlying memory is not manged by
                    # Python so that downstream plugins can access it. Otherwise
                    # the garbage collector will free it when this probe exits.
                    msg_meta = pyds.alloc_nvds_event_msg_meta(user_event_meta)
                    msg_meta.bbox.top = obj_meta.rect_params.top
                    msg_meta.bbox.left = obj_meta.rect_params.left
                    msg_meta.bbox.width = obj_meta.rect_params.width
                    msg_meta.bbox.height = obj_meta.rect_params.height
                    msg_meta.frameId = frame_number
                    msg_meta.trackingId = long_to_uint64(obj_meta.object_id)
                    msg_meta.confidence = obj_meta.confidence
                    msg_meta = generate_event_msg_meta(msg_meta, obj_meta.class_id)

                    user_event_meta.user_meta_data = msg_meta
                    user_event_meta.base_meta.meta_type = pyds.NvDsMetaType.NVDS_EVENT_MSG_META

                    pyds.nvds_add_user_meta_to_frame(frame_meta,
                                                     user_event_meta)
                else:
                    print("Error in attaching event meta to buffer\n")

                is_first_object = False
            try:
                l_obj = l_obj.next
            except StopIteration:
                break
        try:
            l_frame = l_frame.next
        except StopIteration:
            break

    print("Frame Number =", frame_number, "Vehicle Count =",
          obj_counter[PGIE_CLASS_ID_VEHICLE], "Person Count =",
          obj_counter[PGIE_CLASS_ID_PERSON])
    return Gst.PadProbeReturn.OK

# Initializing GStreamer

In [ ]:
platform_info = PlatformInfo()
Gst.init(None)

# Deprecated: following meta_copy_func and meta_free_func
# have been moved to the binding as event_msg_meta_copy_func()
# and event_msg_meta_release_func() respectively.
# Hence, registering and unsetting these callbacks in not needed
# anymore. Please extend the above functions as necessary instead.
# # registering callbacks
# pyds.register_user_copyfunc(meta_copy_func)
# pyds.register_user_releasefunc(meta_free_func)

print("Creating Pipeline \n ")

pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")

# Creating different elements of the stream

## Creating a source element for reading from a file

In [ ]:
print("Creating Source \n ")
source = Gst.ElementFactory.make("filesrc", "file-source")
if not source:
    sys.stderr.write(" Unable to create Source \n")

## Creating a h264 parser as the input file is an elementary h264 stream

In [ ]:
print("Creating H264Parser \n")
h264parser = Gst.ElementFactory.make("h264parse", "h264-parser")
if not h264parser:
    sys.stderr.write(" Unable to create h264 parser \n")

## Using nvdec_h264 for accelerated decoding on GPU

In [ ]:
print("Creating Decoder \n")
decoder = Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")
if not decoder:
    sys.stderr.write(" Unable to create Nvv4l2 Decoder \n")

## Creating a nvstreammux instance to form batches for one or more sources

In [ ]:
streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
if not streammux:
    sys.stderr.write(" Unable to create NvStreamMux \n")

## Setting up nvinfer to run inference on the decoders output
### Note: Behaviour of inference is set through the config file 

In [ ]:
pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
if not pgie:
    sys.stderr.write(" Unable to create pgie \n")

## Using a converter to convert from NV12 to RGBA as required by nvosd

In [ ]:
nvvidconv = Gst.ElementFactory.make("nvvideoconvert", "convertor")
if not nvvidconv:
    sys.stderr.write(" Unable to create nvvidconv \n")

## Create OSD to draw on the converted RGBA buffer

In [ ]:
nvosd = Gst.ElementFactory.make("nvdsosd", "onscreendisplay")
if not nvosd:
    sys.stderr.write(" Unable to create nvosd \n")

## Create a message converter

In [ ]:
msgconv=Gst.ElementFactory.make("nvmsgconv", "nvmsg-converter")
if not msgconv:
    sys.stderr.write(" Unable to create msgconv \n")

## Create a message broker

In [ ]:
msgbroker=Gst.ElementFactory.make("nvmsgbroker", "nvmsg-broker")
if not msgbroker:
    sys.stderr.write(" Unable to create msgbroker \n")

## Create a tee 

In [ ]:
tee=Gst.ElementFactory.make("tee", "nvsink-tee")
if not tee:
    sys.stderr.write(" Unable to create tee \n")

## Creating queues for tee

In [ ]:
queue1=Gst.ElementFactory.make("queue", "nvtee-que1")
if not queue1:
    sys.stderr.write(" Unable to create queue1 \n")

In [ ]:
queue2=Gst.ElementFactory.make("queue", "nvtee-que2")
if not queue2:
    sys.stderr.write(" Unable to create queue2 \n")

## Creating sink as required based on if display is required

In [ ]:
if (no_display) :
    print("Creating FakeSink \n")
    sink = Gst.ElementFactory.make("fakesink", "fakesink")
    if not sink:
        sys.stderr.write(" Unable to create fakesink \n")
else:
    if platform_info.is_integrated_gpu():
        print("Creating nv3dsink \n")
        sink = Gst.ElementFactory.make("nv3dsink", "nv3d-sink")
        if not sink:
            sys.stderr.write(" Unable to create nv3dsink \n")
    else:
        if platform_info.is_platform_aarch64():
            print("Creating nv3dsink \n")
            sink = Gst.ElementFactory.make("nv3dsink", "nv3d-sink")
        else:
            print("Creating EGLSink \n")
            sink = Gst.ElementFactory.make("nveglglessink", "nvvideo-renderer")
        if not sink:
            sys.stderr.write(" Unable to create egl sink \n")

# Setting PGIE and Streammux properties
### Suggested reading: dstest1_pgie_config.txt 

In [ ]:
print("Playing file %s " % input_file)
source.set_property('location', input_file)
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', 1)
streammux.set_property('batched-push-timeout', 33000)
pgie.set_property('config-file-path', PGIE_CONFIG_FILE)
msgconv.set_property('config', MSCONV_CONFIG_FILE)
msgconv.set_property('payload-type', schema_type)
msgbroker.set_property('proto-lib', proto_lib)
msgbroker.set_property('conn-str', conn_str)
if cfg_file is not None:
    msgbroker.set_property('config', cfg_file)
if topic is not None:
    msgbroker.set_property('topic', topic)
msgbroker.set_property('sync', False)

## Adding elements to pipeline

In [ ]:
print("Adding elements to Pipeline \n")
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(tee)
pipeline.add(queue1)
pipeline.add(queue2)
pipeline.add(msgconv)
pipeline.add(msgbroker)
pipeline.add(sink)

## Linking elements of pipline

In [ ]:
print("Linking elements in the Pipeline \n")
source.link(h264parser)
h264parser.link(decoder)

sinkpad = streammux.request_pad_simple("sink_0")
if not sinkpad:
    sys.stderr.write(" Unable to get the sink pad of streammux \n")
srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write(" Unable to get source pad of decoder \n")
srcpad.link(sinkpad)

streammux.link(pgie)
pgie.link(nvvidconv)
nvvidconv.link(nvosd)
nvosd.link(tee)
queue1.link(msgconv)
msgconv.link(msgbroker)
queue2.link(sink)
sink_pad = queue1.get_static_pad("sink")
tee_msg_pad = tee.request_pad_simple('src_%u')
tee_render_pad = tee.request_pad_simple("src_%u")
if not tee_msg_pad or not tee_render_pad:
    sys.stderr.write("Unable to get request pads\n")
tee_msg_pad.link(sink_pad)
sink_pad = queue2.get_static_pad("sink")
tee_render_pad.link(sink_pad)

## Creating an event loop and feeding gstream bus messages to it

In [ ]:
 loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write(" Unable to get sink pad of nvosd \n")
    
osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)

# Starting the pipeline

In [ ]:
print("Starting pipeline \n")

# start play back and listed to events
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass

# Cleaning up post successful run

In [ ]:
pipeline.set_state(Gst.State.NULL)